In [ ]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
columns = {0: 'text', 1: 'ner'}
corpus: Corpus = ColumnCorpus('/content/data',columns,
                              train_file='roberta_TRAIN.txt',
                              dev_file='roberta_TEST.txt',
                              test_file='roberta_TEST.txt')

2022-11-07 04:30:55,632 Reading data from /content/data
2022-11-07 04:30:55,637 Train: /content/IDRIS_train_new_combined (1).txt
2022-11-07 04:30:55,638 Dev: /content/IDRIS_test_new_combined (1).txt
2022-11-07 04:30:55,640 Test: /content/IDRIS_test_new_combined (1).txt


In [ ]:
import pandas as pd
data = [[len(corpus.train),  len(corpus.dev)]]
# Prints out the dataset sizes of train test and development in a table.
pd.DataFrame(data, columns=["Train", "Development"])

,Train,Development
0,14392,2056


In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import WordEmbeddings, StackedEmbeddings, FlairEmbeddings
import torch

# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_label_dictionary(label_type=tag_type)

# 4. initialize fine-tuneable transformer embeddings WITH document context
from flair.embeddings import TransformerWordEmbeddings

embeddings = TransformerWordEmbeddings(
    model='asahi417/tner-xlm-roberta-base-ontonotes5',
    layers="-1",
    subtoken_pooling="first",
    fine_tune=True,
    use_context=True,
)

# 5. initialize bare-bones sequence tagger (no CRF, no RNN, no reprojection)
from flair.models import SequenceTagger

tagger = SequenceTagger(
    hidden_size=768,
    embeddings=embeddings,
    tag_dictionary=tag_dictionary,
    tag_type='ner',
    use_crf=False,
    use_rnn=False,
    reproject_embeddings=False,
)

# 6. initialize trainer with AdamW optimizer
from flair.trainers import ModelTrainer
from torch.optim import AdamW
trainer = ModelTrainer(tagger, corpus)

# 7. run training with XLM parameters (20 epochs, small LR)
from torch.optim.lr_scheduler import OneCycleLR

trainer.train('resources/taggers/ner-english-ontonotes-large',
              learning_rate=5.0e-9,
              mini_batch_size=4,
              mini_batch_chunk_size=1,
              max_epochs=5,
              optimizer=AdamW,
              scheduler=OneCycleLR,
              embeddings_storage_mode='none',
              weight_decay=0.,
              )


2022-11-06 23:54:24,003 Computing label dictionary. Progress:


14392it [00:00, 44109.18it/s]

2022-11-06 23:54:24,339 Dictionary created for label 'ner' with 13 values: CTRY (seen 5068 times), STAT (seen 4437 times), CITY (seen 3731 times), ISL (seen 1014 times), HPOI (seen 269 times), CNTY (seen 248 times), NPOI (seen 153 times), DIST (seen 152 times), NBHD (seen 88 times), CONT (seen 87 times), ST (seen 68 times), OTHR (seen 48 times)


Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

...

2022-11-07 01:57:33,187 Evaluating as a multi-label problem: False
2022-11-07 01:57:33,201 DEV : loss 0.0669732317328453 - f1-score (micro avg)  0.8664
2022-11-07 01:57:33,210 BAD EPOCHS (no improvement): 4
2022-11-07 01:57:33,211 saving best model


2022-11-07 01:57:48,006 ----------------------------------------------------------------------------------------------------
2022-11-07 01:57:48,168 loading file resources/taggers/ner-english-ontonotes-large/best-model.pt
2022-11-07 01:58:01,994 SequenceTagger predicts: Dictionary with 49 tags: O, S-CTRY, B-CTRY, E-CTRY, I-CTRY, S-STAT, B-STAT, E-STAT, I-STAT, S-CITY, B-CITY, E-CITY, I-CITY, S-ISL, B-ISL, E-ISL, I-ISL, S-HPOI, B-HPOI, E-HPOI, I-HPOI, S-CNTY, B-CNTY, E-CNTY, I-CNTY, S-NPOI, B-NPOI, E-NPOI, I-NPOI, S-DIST, B-DIST, E-DIST, I-DIST, S-NBHD, B-NBHD, E-NBHD, I-NBHD, S-CONT, B-CONT, E-CONT, I-CONT, S-ST, B-ST, E-ST, I-ST, S-OTHR, B-OTHR, E-OTHR, I-OTHR


100%|██████████| 33/33 [00:10<00:00,  3.07it/s]

2022-11-07 01:58:14,402 Evaluating as a multi-label problem: False
2022-11-07 01:58:14,427 0.877	0.856	0.8664	0.7698
2022-11-07 01:58:14,428 
Results:
- F-score (micro) 0.8664
- F-score (macro) 0.4629
- Accuracy 0.7698

By class:
              precision    recall  f1-score   support

        STAT     0.9263    0.9362    0.9312        94
        CITY     0.6875    0.7857    0.7333        14
        CNTY     0.8333    1.0000    0.9091         5
        CTRY     0.6000    0.7500    0.6667         4
        HPOI     0.0000    0.0000    0.0000         3
        NPOI     0.0000    0.0000    0.0000         3
        OTHR     0.0000    0.0000    0.0000         2

   micro avg     0.8770    0.8560    0.8664       125
   macro avg     0.4353    0.4960    0.4629       125
weighted avg     0.8261    0.8560    0.8401       125

2022-11-07 01:58:14,432 ----------------------------------------------------------------------------------------------------


{'test_score': 0.8663967611336031,
 'dev_score_history': [0.8225806451612903, 0.8663967611336031],
 'train_loss_history': [0.31706358734405915, 0.2693542941164438],
 'dev_loss_history': [0.08264220505952835, 0.0669732317328453]}

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

input_sentence = 'My name is Eugene, I have seen a live fire at Northern California, spread the word!!!.'
tagger: SequenceTagger = SequenceTagger.load("resources/taggers/ner-english-ontonotes-large/final-model.pt")
sentence: Sentence = Sentence(input_sentence)
tagger.predict(sentence)
print(sentence.to_tagged_string())

2022-11-07 04:34:51,523 loading file resources/taggers/ner-english-ontonotes-large/final-model.pt
2022-11-07 04:35:11,344 SequenceTagger predicts: Dictionary with 49 tags: O, S-CTRY, B-CTRY, E-CTRY, I-CTRY, S-STAT, B-STAT, E-STAT, I-STAT, S-CITY, B-CITY, E-CITY, I-CITY, S-ISL, B-ISL, E-ISL, I-ISL, S-HPOI, B-HPOI, E-HPOI, I-HPOI, S-CNTY, B-CNTY, E-CNTY, I-CNTY, S-NPOI, B-NPOI, E-NPOI, I-NPOI, S-DIST, B-DIST, E-DIST, I-DIST, S-NBHD, B-NBHD, E-NBHD, I-NBHD, S-CONT, B-CONT, E-CONT, I-CONT, S-ST, B-ST, E-ST, I-ST, S-OTHR, B-OTHR, E-OTHR, I-OTHR
Sentence: "My name is Eugene , I have seen a live fire at Northern California , spread the word !!! ." → ["California"/STAT]
